<a href="https://colab.research.google.com/github/filopacio/_python_4_analytics_nlp_project/blob/main/ebola_vs_covid_nlp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



---
## Comparing and contrasting ebola and covid spreading of information on Twitter
---




## Install and Import useful packages 

In [ ]:
!pip3 install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint
!pip install twint
!pip install nest_asyncio
import pandas as pd
import twint 
import nest_asyncio
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer

## Scrape tweets 

**Query for "covid"**


In [5]:
nest_asyncio.apply()
# Configure
c = twint.Config()
c.Search = "covid"
c.Limit = 4000
c.Lang = 'english'
c.Pandas = True
# Run
twint.run.Search(c)
df_c = twint.storage.panda.Tweets_df

**Query for "ebola"**

In [ ]:
nest_asyncio.apply()
# Configure
c = twint.Config()
c.Limit = 4000
c.Search = "ebola"
c.Pandas = True
c.Lang = 'english'
# Run
twint.run.Search(c)
df_e = twint.storage.panda.Tweets_df

## Preprocessing

I created the clean_text function in order to clean the tweets from noisy characters. 

items removed: 
- links
- punctuations/special characters 
- emoticons

Before doing so I also put all the texts in lower case.
I did not remove alphanumeric words to avoid eliminating words like covid19, covid-19 etc.
                

**Text Cleaning**

In [3]:
def clean_text(text):
    text = str(text).lower()
    text = re.sub('https://\S+|www\.\S', '', text)      # remove link
    text = re.sub("['!@#$%^&*()_+<>?:.,;]" , '', text)  # punctuations/special characters
    text = re.sub(re.compile("["                        # emoticon
        u"\U0001F600-\U0001F64F"  
        u"\U0001F300-\U0001F5FF"  
        u"\U0001F680-\U0001F6FF"  
        u"\U0001F1E0-\U0001F1FF"  
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u'\U00010000-\U0010ffff'
        u"\u200d"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\u3030"
        u"\ufe0f"
                           "]+", flags=re.UNICODE), '', text)
    return text

In [ ]:
# SI POTREBBERO CREARE TUTTE LE FUNZIONI DI PRE-PROCESSING  
# PER POI TRASFORMARE IL TESTO TUTTO IN UNA VOLTA ALLA FINE

cl_tweets = [clean_text(c) for c in df_c[df_c.language == 'en'].tweet]

words = [sentence.split() for sentence in cl_tweets]

After being cleaned, each tweet is splitted into single words. 
Therefore, 'words' is a list of lists, where each element is a list of separated strings. 
Now, other pre-processing actions will be performed. 

**Stopwords removal**

In [38]:
def remove_stopwords(text):
    wh_words = ['who', 'what', 'when', 'why', 'how', 'which', 'where', 'whom']
    stop = set(stopwords.words('english'))
    for word in wh_words:
        stop.remove(word)
    text = [[i for i in i.split() if i not in stop] for i in text]
    return text

In [10]:
removed = [remove_stopwords(i) for i in words]

**Lemmatization**

In [11]:
def lemmatize(text):
    lem = WordNetLemmatizer()
    corpus = [[lem.lemmatize(i, pos = 'v') for i in i] for i in text]
    return text

In [12]:
lemmatized = [lemmatize(i) for i in removed]

**Stemming**

In [13]:
def stem(text):
   stemmer = SnowballStemmer(language = 'english')
   text = [[stemmer.stem(i) for i in i] for i in text]
   return text

Final outcome of preprocessing

In [17]:
def preprocess(text):
  text = stem(lemmatize(remove_stopwords(clean_text(text))))
  return text

In [46]:
def word2vec(text):
  vectorizer = CountVectorizer()
  matrix = vectorizer.fit_transform(text)
  return matrix 